<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/DC_OPF_Uncertainty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
!pip install -q gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def status(model):
  if model.Status==2:
    print('Status = optimal')
  elif model.Status==3:
    print('Status = infeasible')

     |████████████████████████████████| 11.5 MB 5.3 MB/s 


# Data

In [ ]:
# generation data
gen = pd.DataFrame({        
       'unit':      [0,    1,     2], 
       'bus':       [0,    1,     2],
       'cost':      [20,   25,    30], 
       'cost_up':   [40,   35,    31], 
       'cost_do':   [0,    15,    29], 
       'pmin':      [0,    0,     0], 
       'pmax':      [150,  150,   100]})

# line data
lin = pd.DataFrame({
       'line': [0,   1,   2], 
       'from': [0,   0,   1],
       'to':   [1,   2,   2], 
       'b':    [10,  10,  10], 
       'cap':  [1000,  1000, 1000]})

# Numbers
ngen = len(gen)
nbus = max(max(lin['from']),max(lin['to']))+1
nlin = len(lin)

# cost_bus and pmax_bus vector
cost_bus = np.zeros(nbus)
cost_up_bus = np.zeros(nbus)
cost_do_bus = np.zeros(nbus)
pmax_bus = np.zeros(nbus)
for g in range(ngen):
  cost_bus[gen.loc[g,'bus']]=gen.loc[g,'cost']
  cost_up_bus[gen.loc[g,'bus']]=gen.loc[g,'cost_up']
  cost_do_bus[gen.loc[g,'bus']]=gen.loc[g,'cost_do']
  pmax_bus[gen.loc[g,'bus']]=gen.loc[g,'pmax']

# cap_line vector
cap_line = np.zeros(nlin)
for l in range(nlin):
  cap_line[lin.loc[l,'line']] = lin.loc[l,'cap']

# Power Transfer Distribution Factors
lin_bus = np.zeros((nlin,nbus))
for l in range(nlin):
  lin_bus[l,lin.loc[l,'from']-1]=1
  lin_bus[l,lin.loc[l,'to']-1]=-1    
matrixX_inv = np.diag(1/lin['b'])
matrixA = np.delete(np.array(lin_bus),0,axis=1)
matrixB = np.linalg.multi_dot([matrixA.T,matrixX_inv,matrixA]) 
ptdf = np.linalg.multi_dot([matrixX_inv,matrixA,np.linalg.inv(matrixB)])
ptdf = np.round(np.insert(ptdf,0,np.zeros((nlin)),axis=1),5)

# postive/negative ptdf
pos_neg = np.zeros((nlin,nbus))
for l in range(nlin):
  for b in range(nbus):
    if ptdf[l,b] > 0:
      pos_neg[l,b]=1
    elif ptdf[l,b] < 0:
      pos_neg[l,b]=-1

# scenario generation
nscen = 1000
demand = np.array([100,100,100])
var_10 = np.array([10,10,10])
var_20 = np.array([20,20,20])
var_25 = np.array([25,25,25])
var_30 = np.array([30,30,30])
scen_10 = 100 + 10*np.random.uniform(-1,1,size=(nscen,nbus))
scen_20 = 100 + 20*np.random.uniform(-1,1,size=(nscen,nbus))
scen_25 = 100 + 25*np.random.uniform(-1,1,size=(nscen,nbus))
scen_30 = 100 + 30*np.random.uniform(-1,1,size=(nscen,nbus))

# Deterministic DC-OPF

$$
\begin{align}
\underset{g_n}{\min} \quad & \sum_n c_n g_n \\
\text{s.t.} \quad & \sum_n g_n = \sum_n \hat{d}_n\\
& 0 \leq g_n \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_n b_{ln}(g_n-\hat{d}_n) \leq \overline{f}_l, \quad \forall l
\end{align}
$$

In [ ]:
def dc_opf_det(demand):
  # model
  m = gp.Model() 
  # variables
  gen = m.addMVar(nbus,name='gen')
  # objective function
  m.setObjective(cost_bus @ gen, GRB.MINIMIZE)
  # constraints
  m.addConstr( gen.sum() == demand.sum())
  m.addConstr(gen <= pmax_bus)
  m.addConstr(ptdf @ gen >= -cap_line + ptdf @ demand)
  m.addConstr(ptdf @ gen <= cap_line + ptdf @ demand)
  # solve
  m.write('det.lp')
  m.setParam('OutputFlag',0)
  m.optimize()
  status(m)
  # results
  print('cost =',m.ObjVal)
  print('dispatch =',gen.X)
  return gen.X
gen_det = dc_opf_det(demand)

Status = optimal
cost = 6750.0
dispatch = [150. 150.   0.]


# Real time operation

$$
\begin{align}
\underset{r^u_n,r^d_n}{\min} \quad & \sum_n c_n g^*_n + c^u_n r^u_n + c^d_n r^d_n \\
\text{s.t.} \quad & \sum_n g^*_n+r^u_n-r^d_n = \sum_n \tilde{d}_n\\
& 0 \leq g^*_n + r^u_n-r^d_n \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_n b_{ln}(g^*_n + r^u_n-r^d_n -\tilde{d}_n) \leq \overline{f}_l, \quad \forall l \\
& r^u_n, r^d_n\geq 0
\end{align}
$$

In [ ]:
def real_time(demand,dispatch):
  #print('Solving real_time for demand=',demand)
  # model
  m = gp.Model()
  # variables
  #res = m.addMVar(nbus,lb=-GRB.INFINITY)
  res_up = m.addMVar(nbus)
  res_do = m.addMVar(nbus)
  # objective function
  m.setObjective(cost_bus @ dispatch + cost_up_bus @ res_up - cost_do_bus @ res_do, GRB.MINIMIZE)
  # constraints
  m.addConstr(dispatch.sum() + res_up.sum() - res_do.sum() == demand.sum())
  m.addConstr(dispatch + res_up - res_do >= 0)
  m.addConstr(dispatch + res_up - res_do <= pmax_bus)
  m.addConstr(ptdf @ dispatch + ptdf @ res_up - ptdf @ res_do >= -cap_line + ptdf @ demand)
  m.addConstr(ptdf @ dispatch + ptdf @ res_up - ptdf @ res_do <= cap_line + ptdf @ demand)
  #m.addConstr(res >= -res_do)
  #m.addConstr(res <= res_up)
  # solve
  m.setParam('OutputFlag',0)
  m.optimize()
  # status(m)
  # results
  #print('reserves =',res.X)
  #print('reserves_up =',res_up.X)
  #print('reserves_do =',res_do.X)
  return m.ObjVal

In [ ]:
def evaluate_scenarios(dispatch,scen):
  v_cost = []
  for s in range(nscen):
    obj = real_time(scen[s,:],dispatch)
    v_cost.append(obj)    
  print('minimum cost =',np.min(v_cost))
  print('average cost =',np.mean(v_cost))
  print('maximum cost =',np.max(v_cost))
  #plt.figure(facecolor='white')
  #plt.hist(v_cost, 10, facecolor='blue', alpha=0.5)
  #plt.show()  

# Evaluate deterministic dispatch for var = 10

In [ ]:
evaluate_scenarios(gen_det,scen_10)

minimum cost = 6382.704098898973
average cost = 6822.270823258412
maximum cost = 7554.171046329571


# Evaluate deterministic dispatch for var = 20

In [ ]:
evaluate_scenarios(gen_det,scen_20)

minimum cost = 5975.656890488153
average cost = 6884.494057704674
maximum cost = 8428.903774265413


# Evaluate deterministic dispatch for var = 30

In [ ]:
evaluate_scenarios(gen_det,scen_30)

minimum cost = 5495.806616590174
average cost = 6962.044270331796
maximum cost = 9123.783571635482


# Robust (no recourse)

$d_n = \hat{d}_n - \overline{e}_n \eta_n$ with $\eta_n \in [-1,1]$

$$
\begin{align}
\underset{g_n}{\min} \quad & \sum_n c_n g_n  \\
\text{s.t.} \quad & \sum_n g_n \geq \underset{\eta_n\in[-1,1]}{\max} \sum_n \hat{d}_n - \overline{e}_n \eta_n \\
& 0 \leq g_n \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \underset{\eta_n\in[-1,1]}{\min} \sum_n b_{ln}(g_n -\hat{d}_n + \overline{e}_n \eta_n), \quad \forall l \\
&  \underset{\eta_n\in[-1,1]}{\max} \sum_n b_{ln}(g_n -\hat{d}_n + \overline{e}_n \eta_n) \leq \overline{f}_l, \quad \forall l 
\end{align}
$$

Reformulation 

$$
\begin{align}
\underset{g_n}{\min} \quad & \sum_n c_n g_n  \\
\text{s.t.} \quad & \sum_n g_n \geq \sum_n \hat{d}_n + \overline{e}_n \\
& 0 \leq g_n \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_{n\in\mathcal{N}^+_l} b_{ln}(g_n -\hat{d}_n - \overline{e}_n) + \sum_{n\in\mathcal{N}^-_l} b_{ln}(g_n -\hat{d}_n + \overline{e}_n) , \quad \forall l \\
& \sum_{n\in\mathcal{N}^+_l} b_{ln}(g_n -\hat{d}_n + \overline{e}_n) + \sum_{n\in\mathcal{N}^-_l} b_{ln}(g_n -\hat{d}_n - \overline{e}_n) \leq \overline{f}_l, \quad \forall l \\
\end{align}
$$

where $\mathcal{N}^+_l$ set of nodes such that $b_{ln}>0$ and $\mathcal{N}^-_l$ set of nodes such that $b_{ln}\lt 0$ 

In [ ]:
def dc_opf_robust(demand,var):
  # model
  m = gp.Model()  
  # variables
  gen = m.addMVar(nbus,name='gen')
  # objective function
  m.setObjective(cost_bus @ gen, GRB.MINIMIZE)
  # constraints
  m.addConstr(gen.sum() >= demand.sum() + var.sum())
  m.addConstr(gen <= pmax_bus)
  m.addConstr(ptdf @ gen >= -cap_line + ptdf @ demand + np.multiply(ptdf,pos_neg) @ var)
  m.addConstr(ptdf @ gen <= cap_line + ptdf @ demand - np.multiply(ptdf,pos_neg) @ var)
  # solve
  m.setParam('OutputFlag',0)
  m.write('robust.lp')
  m.optimize()
  status(m)
  # results
  print('dispatch =',gen.X)
  return gen.X

# Evaluate robust dispatch for var = 10

In [ ]:
gen_robust = dc_opf_robust(demand,var_10)
evaluate_scenarios(gen_robust,scen_10)

Status = optimal
dispatch = [150. 150.  30.]
minimum cost = 6412.704098898973
average cost = 6843.593336088337
maximum cost = 7532.289043340566


# Evaluate robust dispatch for var = 20

In [ ]:
gen_robust = dc_opf_robust(demand,var_20)
evaluate_scenarios(gen_robust,scen_20)

Status = optimal
dispatch = [175.00015  84.99985 100.     ]
minimum cost = 6877.945811831988
average cost = 7740.967613654493
maximum cost = 8592.623488592011


# Evaluate robust dispatch for var = 30

In [ ]:
gen_robust = dc_opf_robust(demand,var_30)
evaluate_scenarios(gen_robust,scen_30)

Status = optimal
dispatch = [150. 150.  90.]
minimum cost = 5585.281967999951
average cost = 7014.361557623672
maximum cost = 9174.22307933641


If we consider a budget $\Gamma$ we have to use the duals

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\max} \quad & -\sum_n \overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \alpha \\
& \eta^+_n \leq 1: \beta^+_n, \quad \forall n\\
& \eta^-_n \leq 1: \beta^-_n, \quad \forall n \\
& \eta^+_n,\eta^-_n\geq 0 
\end{align}
$$

The dual is

$$
\begin{align}
\underset{\alpha, \beta^+_n,\beta^-_n}{\min} \quad & \alpha\Gamma + \sum_n  \beta^+_n+\beta^-_n  \\
\text{s.t.} \quad & \overline{e}_n + \alpha + \beta^+_n \geq 0, \quad \forall n \\
& -\overline{e}_n + \alpha + \beta^-_n \geq 0, \quad \forall n \\
& \alpha, \beta^+_n,\beta^-_n\geq 0
\end{align}
$$

With the other constraint

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\min} \quad & \sum_n b_{ln}\overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \gamma_l \\
& \eta^+_n \leq 1: \delta^+_{ln}, \quad \forall n\\
& \eta^-_n \leq 1: \delta^-_{ln}, \quad \forall n \\
& \eta^+_n,\eta^-_n\geq 0
\end{align}
$$

The dual is

$$
\begin{align}
\underset{\gamma_l, \delta^+_{ln},\delta^-_{ln}}{\max} \quad & -\gamma_l\Gamma - \sum_n  \delta^+_{ln}+\delta^-_{ln}  \\
\text{s.t.} \quad & b_{ln}\overline{e}_n + \gamma_l + \delta^+_{ln} \geq 0, \quad \forall n \\
& -b_{ln}\overline{e}_n + \gamma_l + \delta^-_{ln} \geq 0, \quad \forall n \\
& \gamma_l, \delta^+_{ln},\delta^-_{ln}\geq 0
\end{align}
$$

Last constraint

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\max} \quad & \sum_n b_{ln}\overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \epsilon_l \\
& \eta^+_n \leq 1: \zeta^+_{ln}, \quad \forall n\\
& \eta^-_n \leq 1: \zeta^-_{ln}, \quad \forall n\\
& \eta^+_n,\eta^-_n\geq 0
\end{align}
$$

The dual is

$$
\begin{align}
\underset{\epsilon_l, \zeta^+_{ln},\zeta^-_{ln}}{\min} \quad & \epsilon_l\Gamma + \sum_n  \zeta^+_{ln}+\zeta^-_{ln}  \\
\text{s.t.} \quad & -b_{ln}\overline{e}_n + \epsilon_l + \zeta^+_{ln} \geq 0, \quad \forall n \\
& b_{ln}\overline{e}_n + \epsilon_l + \zeta^-_{ln} \geq 0, \quad \forall n \\
& \epsilon_l, \zeta^+_{ln},\zeta^-_{ln}\geq 0
\end{align}
$$

We replace everything and obtain

$$
\begin{align}
\underset{g_n,\alpha,\beta^+_n,\beta^-_n,\gamma_l,\delta^+_{ln},\delta^-_{ln},\epsilon_l,\zeta^+_{ln},\zeta^-_{ln}}{\min} \quad & \sum_n c_n g_n  \\
\text{s.t.} \quad & 0 \leq g_n \leq \overline{g}_n, \quad \forall n\\
& \sum_n g_n \geq \alpha\Gamma + \sum_n  \hat{d}_n + \beta^+_n+\beta^-_n \\
& -\overline{f}_l \leq -\gamma_l\Gamma - \sum_n \delta^+_{ln}+\delta^-_{ln} - b_{ln}(g_n-\hat{d}_n), \quad \forall l \\
&  \epsilon_l\Gamma + \sum_n  \zeta^+_{ln}+\zeta^-_{ln}+b_{ln}(g_n-\hat{d}_n) \leq \overline{f}_l, \quad \forall l \\
& \overline{e}_n + \alpha + \beta^+_n \geq 0, \quad \forall n \\
& -\overline{e}_n + \alpha + \beta^-_n \geq 0, \quad \forall n \\
& b_{ln}\overline{e}_n + \gamma_l + \delta^+_{ln} \geq 0, \quad \forall l,n \\
& -b_{ln}\overline{e}_n + \gamma_l + \delta^-_{ln} \geq 0, \quad \forall l,n \\
& -b_{ln}\overline{e}_n + \epsilon_l + \zeta^+_{ln} \geq 0, \quad \forall l,n \\
& b_{ln}\overline{e}_n + \epsilon_l + \zeta^-_{ln} \geq 0, \quad \forall l,n \\
& \alpha,\beta^+_n,\beta^-_n,\gamma_l,\delta^+_{ln},\delta^-_{ln},\epsilon_l,\zeta^+_{ln},\zeta^-_{ln} \geq 0
\end{align}
$$

In [ ]:
def dc_opf_robust_budget(demand,var,budget):
  # model
  m = gp.Model()  
  # variables
  gen = m.addMVar(nbus)
  alpha = m.addVar()
  beta_up = m.addMVar(nbus)
  beta_do = m.addMVar(nbus)
  gamma = m.addMVar(nlin)
  delta_up = m.addMVar((nlin,nbus))
  delta_do = m.addMVar((nlin,nbus))
  epsilon = m.addMVar(nlin)
  zeta_up = m.addMVar((nlin,nbus))
  zeta_do = m.addMVar((nlin,nbus))
  # objective function
  m.setObjective(cost_bus @ gen, GRB.MINIMIZE)
  # constraints
  m.addConstr(gen <= pmax_bus)
  m.addConstr(gen.sum() >= alpha*budget + demand.sum() + beta_up.sum() + beta_do.sum() )  
  m.addConstrs(-cap_line[l] <= - budget*gamma[l] - delta_up[l,:].sum() - delta_do[l,:].sum() + ptdf[l,:] @ gen - ptdf[l,:] @ demand for l in range(nlin))
  m.addConstrs(budget*epsilon[l] + zeta_up[l,:].sum() + zeta_do[l,:].sum() + ptdf[l,:] @ gen - ptdf[l,:] @ demand <= cap_line[l] for l in range(nlin))
  m.addConstrs(var[n] + alpha + beta_up[n] >= 0 for n in range(nbus))
  m.addConstrs(-var[n] + alpha + beta_do[n] >= 0 for n in range(nbus))
  m.addConstrs(var[b]*ptdf[:,b] + gamma + delta_up[:,b] >= 0 for b in range(nbus))
  m.addConstrs(-var[b]*ptdf[:,b] + gamma + delta_do[:,b] >= 0 for b in range(nbus))
  m.addConstrs(-var[b]*ptdf[:,b] + epsilon + zeta_up[:,b] >= 0 for b in range(nbus))
  m.addConstrs(var[b]*ptdf[:,b] + epsilon + zeta_do[:,b] >= 0 for b in range(nbus))
  # solve
  m.setParam('OutputFlag',0)
  m.write('robust_ldr.lp')
  m.optimize()
  status(m)
  # results
  print('dispatch =',gen.X)
  return gen.X

# Evaluate robust dispatch (Gamma=2, var=10)

In [ ]:
gen_robust = dc_opf_robust_budget(demand,var_10,2)
evaluate_scenarios(gen_robust,scen_10)

Status = optimal
dispatch = [150. 150.  20.]
minimum cost = 6402.704098898973
average cost = 6833.71171739833
maximum cost = 7534.171046329571


# Evaluate robust dispatch (Gamma=2, var=20)

In [ ]:
gen_robust = dc_opf_robust_budget(demand,var_20,2)
evaluate_scenarios(gen_robust,scen_20)

Status = optimal
dispatch = [155.00015  84.99985 100.     ]
minimum cost = 6481.4263800467525
average cost = 7406.814420124009
maximum cost = 8722.11167142717


# Evaluate robust dispatch (Gamma=2, var=30)

In [ ]:
gen_robust = dc_opf_robust_budget(demand,var_30,2)
evaluate_scenarios(gen_robust,scen_30)

Status = infeasible


GurobiError: ignored

# Evaluate robust dispatch (Gamma=1, var=10)

In [ ]:
gen_robust = dc_opf_robust_budget(demand,var_10,1)
evaluate_scenarios(gen_robust,scen_10)

Status = optimal
dispatch = [165.00045  44.99955 100.     ]
minimum cost = 6715.041700270952
average cost = 7219.658964650112
maximum cost = 8032.622783500859


# Evaluate robust dispatch (Gamma=1, var=20)

In [ ]:
gen_robust = dc_opf_robust_budget(demand,var_20,1)
evaluate_scenarios(gen_robust,scen_20)

Status = optimal
dispatch = [155.00015  64.99985 100.     ]
minimum cost = 6281.426380046753
average cost = 7331.68786194085
maximum cost = 8922.11167142717


# Evaluate robust dispatch (Gamma=1, var=30)

In [ ]:
gen_robust = dc_opf_robust_budget(demand,var_30,1)
evaluate_scenarios(gen_robust,scen_30)

Status = optimal
dispatch = [144.99985  85.00015 100.     ]
minimum cost = 5860.008232108621
average cost = 7402.270964695413
maximum cost = 9839.711481954711


# Linear decision rules

We define $\Omega = \sum_n \overline{e}_n\eta_n$, $\underline{\Omega}=-\sum_n\overline{e}_n$, and $\overline{\Omega}=\sum_n\overline{e}_n$. We use a linear decision rule so that the output of each unit is determined by $g_n-\beta_n\Omega$. We first minimize the expected cost

$$
\begin{align}
\underset{g_n,\beta_n}{\min} \quad & \mathbb{E} \left[ \sum_n c_n g_n + c^u_n\max(0,-\beta_n \Omega) -c^d_n \max(0,\beta_n\Omega) \right] \\
\text{s.t.} \quad & \sum_n g_n-\beta_n\Omega = \sum_n \hat{d}_n - \overline{e}_n\eta_n \\
& 0 \leq g_n - \beta_n\Omega \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \underset{\eta_n\in[-1,1]}{\min} \sum_n b_{ln}(g_n - \beta_n\Omega -\hat{d}_n + \overline{e}_n \eta_n), \quad \forall l \\
&  \underset{\eta_n\in[-1,1]}{\max} \sum_n b_{ln}(g_n - \beta_n\Omega -\hat{d}_n + \overline{e}_n \eta_n) \leq \overline{f}_l, \quad \forall l \\
& \beta_n \geq 0
\end{align}
$$

We can reformulate as 

$$
\begin{align}
\underset{g_n,\beta_n}{\min} \quad &  \mathbb{E} \left[ \sum_n c_n g_n + c^u_n\max(0,-\beta_n \Omega) -c^d_n \max(0,\beta_n\Omega) \right] \\\
\text{s.t.} \quad & \sum_n g_n = \sum_n \hat{d}_n \\
& \sum_n \beta_n = 1 \\
& 0 \leq g_n - \underset{\Omega\in[\underline{\Omega},\overline{\Omega}]}{\max}\beta_n\Omega, \quad \forall n\\
& g_n - \underset{\Omega\in[\underline{\Omega},\overline{\Omega}]}{\min}\beta_n\Omega \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \underset{\eta_n\in[-1,1]}{\min} \sum_n b_{ln}(g_n - \beta_n\Omega -\hat{d}_n + \overline{e}_n \eta_n), \quad \forall l \\
&  \underset{\eta_n\in[-1,1]}{\max} \sum_n b_{ln}(g_n - \beta_n\Omega -\hat{d}_n + \overline{e}_n \eta_n) \leq \overline{f}_l, \quad \forall l \\
& \beta_n \geq 0
\end{align}
$$

And therefore

$$
\begin{align}
\underset{g_n,\beta_n}{\min} \quad &  \sum_n c_n g_n + 0.25 \overline{\Omega} (c^u_n-c^d_n) \beta_n \\
\text{s.t.} \quad & \sum_n g_n = \sum_n \hat{d}_n \\
& \sum_n \beta_n = 1 \\
& 0 \leq g_n - \beta_n\overline{\Omega}, \quad \forall n\\
& g_n - \beta_n\underline{\Omega} \leq \overline{g}_n, \quad \forall n\\
& -\overline{f}_l \leq \sum_{n\in\mathcal{N}^+_l} b_{ln}(g_n - \beta_n\overline{\Omega} -\hat{d}_n - \overline{e}_n) + \sum_{n\in\mathcal{N}^-_l} b_{ln}(g_n - \beta_n\underline{\Omega} -\hat{d}_n + \overline{e}_n), \quad \forall l \\
&  \sum_{n\in\mathcal{N}^+_l} b_{ln}(g_n - \beta_n\underline{\Omega} -\hat{d}_n + \overline{e}_n) + \sum_{n\in\mathcal{N}^-_l} b_{ln}(g_n - \beta_n\overline{\Omega} -\hat{d}_n - \overline{e}_n ) \leq \overline{f}_l, \quad \forall l \\
& \beta_n \geq 0
\end{align}
$$

In [ ]:
def dc_opf_ldr(demand,var):
  # model
  m = gp.Model()
  # variables
  gen = m.addMVar(nbus,name='gen')
  beta = m.addMVar(nbus,name='beta')
  # objective function
  m.setObjective(cost_bus @ gen + 0.25 * var.sum() * cost_up_bus @ beta - 0.25 * var.sum() * cost_do_bus @ beta, GRB.MINIMIZE)
  # constraints
  m.addConstr(gen.sum() == demand.sum())
  m.addConstr(beta.sum() == 1)
  m.addConstr(0 <= gen - var.sum()*beta)
  m.addConstr(gen + var.sum()*beta <= pmax_bus)
  m.addConstr(ptdf @ gen - var.sum() * ptdf @ beta >= -cap_line + ptdf @ demand + np.multiply(ptdf,pos_neg) @ var)
  m.addConstr(ptdf @ gen - var.sum() * ptdf @ beta <= cap_line + ptdf @ demand - np.multiply(ptdf,pos_neg) @ var)
  # solve
  m.setParam('OutputFlag',0)
  m.write('ldr.lp')
  m.optimize()
  status(m) 
  # results
  print('dispatch =',gen.X,beta.X)
  return gen.X

# Evaluate LDR dispatch with var = 10

In [ ]:
gen_robust_ldr = dc_opf_ldr(demand,var_10)
evaluate_scenarios(gen_robust_ldr,scen_10)

Status = optimal
dispatch = [150. 135.  15.] [0.  0.5 0.5]
minimum cost = 6247.704098898973
average cost = 6842.709872606447
maximum cost = 7629.171046329571


# Evaluate LDR dispatch with var = 20

In [ ]:
gen_robust_ldr = dc_opf_ldr(demand,var_20)
evaluate_scenarios(gen_robust_ldr,scen_20)

Status = optimal
dispatch = [150. 120.  30.] [0.  0.5 0.5]
minimum cost = 5651.602242301942
average cost = 6912.971384698708
maximum cost = 8545.016633640025


# Evaluate LDR dispatch with var = 30


In [ ]:
gen_robust_ldr = dc_opf_ldr(demand,var_30)
evaluate_scenarios(gen_robust_ldr,scen_30)

Status = optimal
dispatch = [150. 105.  45.] [0.  0.5 0.5]
minimum cost = 5090.281967999951
average cost = 7009.086358431071
maximum cost = 9572.165785406014


First primal

$$
\begin{align}
\underset{\eta^+_m,\eta^-_m}{\max} \quad & \beta_n\sum_m \overline{e}_m(\eta^+_m-\eta^-_m)  \\
\text{s.t.} \quad & \sum_m \eta^+_m+\eta^-_m \leq \Gamma: \tau_n \\
& \eta^+_m \leq 1: \rho^+_{nm}, \quad \forall m\\
& \eta^-_m \leq 1: \rho^-_{nm}, \quad \forall m\\
& \eta^+_m,\eta^-_m\geq 0
\end{align}
$$


First dual

$$
\begin{align}
\underset{\tau_n,\rho^+_{nm},\rho^-_{nm}}{\min} \quad & \Gamma\tau_n + \sum_m \rho^+_{nm} + \rho^-_{nm}  \\
\text{s.t.} \quad & -\beta_n\overline{e}_m + \tau_n + \rho^+_{nm} \geq 0, \quad \forall m \\
& \beta_n\overline{e}_m + \tau_n + \rho^-_{nm} \geq 0, \quad \forall m\\
& \tau,\rho^+_{nm},\rho^-_{nm} \geq 0
\end{align}
$$

Second primal

$$
\begin{align}
\underset{\eta^+_{m},\eta^-_{m}}{\min} \quad & \beta_n \sum_m \overline{e}_m (\eta^+_{m}-\eta^-_{m})  \\
\text{s.t.} \quad & \sum_m \eta^+_{m}+\eta^-_{m} \leq \Gamma: \kappa_n \\
& \eta^+_m \leq 1: \mu^+_{nm}, \quad \forall m\\
& \eta^-_m \leq 1: \mu^-_{nm}, \quad \forall m\\
& \eta^+_m,\eta^-_m\geq 0
\end{align}
$$

Second dual

$$
\begin{align}
\underset{\kappa_n,\mu^+_{nm},\mu^-_{nm}}{\max} \quad & -\Gamma\kappa_n - \sum_m \mu^+_{nm} + \mu^-_{nm}  \\
\text{s.t.} \quad & \beta_n\overline{e}_m + \kappa_n + \mu^+_{nm} \geq 0, \quad \forall m \\
& -\beta_n\overline{e}_m + \kappa_n + \mu^-_{nm} \geq 0, \quad \forall m\\
& \kappa_n,\mu^+_{nm},\mu^-_{nm} \geq 0
\end{align}
$$

Third primal

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\min} \quad & \sum_n -b_{ln}\beta_n\sum_m \overline{e}_m (\eta^+_m-\eta^-_m) +b_{ln}\overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \gamma_l \\
& \eta^+_n \leq 1: \delta^+_{ln}, \quad \forall n\\
& \eta^-_n \leq 1: \delta^-_{ln}, \quad \forall n\\
& \eta^+_n,\eta^-_n\geq 0
\end{align}
$$

Third dual

$$
\begin{align}
\underset{\gamma_l, \delta^+_{ln},\delta^-_{ln}}{\max} \quad & - \gamma_l\Gamma - \sum_n  \delta^+_{ln}+\delta^-_{ln}  \\
\text{s.t.} \quad & -\overline{e}_n \left(-b_{ln} + \sum_m b_{lm}\beta_m \right) + \gamma_l + \delta^+_{ln} \geq 0, \quad \forall n \\
& \overline{e}_n \left(-b_{ln} + \sum_m b_{lm}\beta_m \right) + \gamma_l + \delta^-_{ln} \geq 0, \quad \forall n \\
& \gamma_l, \delta^+_{ln},\delta^-_{ln}\geq 0
\end{align}
$$

Four primal

$$
\begin{align}
\underset{\eta^+_n,\eta^-_n}{\max} \quad & \sum_n -b_{ln}\beta_n\sum_m \overline{e}_m (\eta^+_m-\eta^-_m) +b_{ln}\overline{e}_n (\eta^+_n-\eta^-_n)  \\
\text{s.t.} \quad & \sum_n \eta^+_n+\eta^-_n \leq \Gamma: \epsilon_l \\
& \eta^+_n \leq 1: \zeta^+_{ln}, \quad \forall n\\
& \eta^-_n \leq 1: \zeta^-_{ln}, \quad \forall n\\
& \eta^+_n,\eta^-_n\geq 0
\end{align}
$$

Four dual

$$
\begin{align}
\underset{\epsilon_l, \zeta^+_{ln},\zeta^-_{ln}}{\min} \quad &  \epsilon_l\Gamma + \sum_n  \zeta^+_{ln}+\zeta^-_{ln}  \\
\text{s.t.} \quad & \sum_n b_{ln}\beta_n - \lambda_l = 0 \\
& \overline{e}_n \left(-b_{ln} + \sum_m b_{lm}\beta_m \right) + \epsilon_l + \zeta^+_{ln} \geq 0, \quad \forall n \\
& -\overline{e}_n \left(-b_{ln} + \sum_m b_{lm}\beta_m \right) + \epsilon_l + \zeta^-_{ln} \geq 0, \quad \forall n \\
& \epsilon_l, \zeta^+_{ln},\zeta^-_{ln}\geq 0
\end{align}
$$

By using the dual formulation we obtain

$$
\begin{align}
\underset{g_n,\beta_n,\tau_n,\rho^+_{nm},\rho^-_{nm},\kappa_n,\mu^+_{nm},\mu^-_{nm},\gamma_l, \delta^+_{ln},\delta^-_{ln},\epsilon_l, \zeta^+_{ln},\zeta^-_{ln}}{\min} \quad &  \sum_n c_n g_n + 0.25 \overline{\Omega} (c^u_n-c^d_n) beta_n  \\
\text{s.t.} \quad & \sum_n g_n = \sum_n \hat{d}_n \\
& \sum_n \beta_n = 1 \\
& 0 \leq g_n - \Gamma\tau_n - \sum_m \rho^+_{nm} + \rho^-_{nm}, \quad \forall n\\
& -\beta_n\overline{e}_m + \tau_n + \rho^+_{nm} \geq 0, \quad \forall n,m \\
& \beta_n\overline{e}_m + \tau_n + \rho^-_{nm} \geq 0, \quad \forall n,m\\
& g_n + \Gamma\kappa_n + \sum_m \mu^+_{nm} + \mu^-_{nm} \leq \overline{g}_n, \quad \forall n\\
& \beta_n\overline{e}_m + \kappa_n + \mu^+_{nm} \geq 0, \quad \forall n,m \\
& -\beta_n\overline{e}_m + \kappa_n + \mu^-_{nm} \geq 0, \quad \forall n,m\\
& -\overline{f}_l \leq \sum_n b_{ln}(g_n-\hat{d}_n) -\gamma_l\Gamma - \sum_n \delta^+_{ln} + \delta^-_{ln}, \quad \forall l \\
& -\overline{e}_n \left(-b_{ln} + \sum_m b_{lm}\beta_m \right) + \gamma_l + \delta^+_{ln} \geq 0, \quad \forall n,l \\
& \overline{e}_n \left(-b_{ln} + \sum_m b_{lm}\beta_m \right) + \gamma_l + \delta^-_{ln} \geq 0, \quad \forall n,l \\
& \sum_n b_{ln}(g_n-\hat{d}_n) + \epsilon_l\Gamma + \sum_n \zeta^+_{ln}+\zeta^-_{ln} \leq \overline{f}_l, \quad \forall l \\
& \overline{e}_n \left(-b_{ln} + \sum_m b_{lm}\beta_m \right) + \epsilon_l + \zeta^+_{ln} \geq 0, \quad \forall n,l \\
& -\overline{e}_n \left(-b_{ln} + \sum_m b_{lm}\beta_m \right) + \epsilon_l + \zeta^-_{ln} \geq 0, \quad \forall n,l \\
& g_n,\beta_n, \tau_n,\rho^+_{nm}\rho^-_{nm}, \kappa_n,\mu^+_{nm},\mu^-_{nm},\gamma_l, \delta^+_{ln},\delta^-_{ln}, \epsilon_l, \zeta^+_{ln},\zeta^-_{ln} \geq 0
\end{align}
$$

In [ ]:
def dc_opf_ldr_budget(demand,var,budget):
  # model
  m = gp.Model()
  m.Params.Threads = 1
  # variables
  gen = m.addMVar(nbus,name='gen')
  beta = m.addMVar(nbus,name='beta')
  tau = m.addMVar(nbus,name='tau')
  rho_up = m.addMVar((nbus,nbus),name='rho_up')
  rho_do = m.addMVar((nbus,nbus),name='rho_do')
  kappa = m.addMVar(nbus,name='kappa')
  mu_up = m.addMVar((nbus,nbus),name='mu_up')
  mu_do = m.addMVar((nbus,nbus),name='mu_do')  
  gamma = m.addMVar(nlin,name='gamma')  
  delta_up = m.addMVar((nlin,nbus),name='delta_up')
  delta_do = m.addMVar((nlin,nbus),name='delta_do')  
  epsilon = m.addMVar(nlin,name='epsilon')
  zeta_up = m.addMVar((nlin,nbus),name='zeta_up')
  zeta_do = m.addMVar((nlin,nbus),name='zeta_do')
  # objective function
  m.setObjective(cost_bus @ gen + 0.25 * var.sum() * (cost_up_bus @ beta) - 0.25 * var.sum() * (cost_do_bus @ beta), GRB.MINIMIZE)  
  # constraints
  m.addConstr(gen.sum() == demand.sum())
  m.addConstr(beta.sum() == 1)
  m.addConstrs(0 <= gen[n] - budget*tau[n] - rho_up[n,:].sum() - rho_do[n,:].sum() for n in range(nbus))
  m.addConstrs(-beta[n]*var[m] + tau[n] + rho_up[n,m] >= 0 for n in range(nbus) for m in range(nbus))
  m.addConstrs(beta[n]*var[m] + tau[n] + rho_do[n,m] >= 0 for n in range(nbus) for m in range(nbus))
  m.addConstrs(gen[n] + budget*kappa[n] + mu_up[n,:].sum() + mu_do[n,:].sum() <= pmax_bus[n] for n in range(nbus))
  m.addConstrs(beta[n]*var[m] + kappa[n] + mu_up[n,m] >= 0 for n in range(nbus) for m in range(nbus))
  m.addConstrs(-beta[n]*var[m] + kappa[n] + mu_do[n,m] >= 0 for n in range(nbus) for m in range(nbus))
  m.addConstrs(-cap_line[l] <= sum(ptdf[l,n]*(gen[n]-demand[n]) for n in range(nbus)) -budget*gamma[l] - delta_up[l,:].sum() - delta_do[l,:].sum() for l in range(nlin))
  m.addConstrs(-var[n]*(-ptdf[l,n] + sum(ptdf[l,m]*beta[m] for m in range(nbus))) + gamma[l] + delta_up[l,n] >= 0 for n in range(nbus) for l in range(nlin))
  m.addConstrs(var[n]*(-ptdf[l,n] + sum(ptdf[l,m]*beta[m] for m in range(nbus))) + gamma[l] + delta_do[l,n] >= 0 for n in range(nbus) for l in range(nlin))
  m.addConstrs(sum(ptdf[l,n]*(gen[n]-demand[n]) for n in range(nbus)) + budget*epsilon[l] + zeta_up[l,:].sum() + zeta_do[l,:].sum() <= cap_line[l] for l in range(nlin)) 
  m.addConstrs(var[n]*(-ptdf[l,n] + sum(ptdf[l,m]*beta[m] for m in range(nbus))) + epsilon[l] + zeta_up[l,n] >= 0 for n in range(nbus) for l in range(nlin))
  m.addConstrs(-var[n]*(-ptdf[l,n] + sum(ptdf[l,m]*beta[m] for m in range(nbus))) + epsilon[l] + zeta_do[l,n] >= 0 for n in range(nbus) for l in range(nlin))  
  # solve
  m.setParam('OutputFlag',0)  
  m.optimize()
  print('obj =',m.ObjVal)
  status(m)   
  m.write('ldr_budget.lp')
  # results
  print('dispatch =',gen.X,beta.X)
  return gen.X

# Evaluate LDR dispatch (Gamma=2, var=10)

In [ ]:
gen_ldr_budget = dc_opf_ldr_budget(demand,var_10,2)
evaluate_scenarios(gen_ldr_budget,scen_10)

# Evaluate LDR dispatch (Gamma=2, var=20)

In [ ]:
gen_ldr_budget = dc_opf_ldr_budget(demand,var_20,2)
evaluate_scenarios(gen_ldr_budget,scen_20)

# Evaluate LDR dispatch (Gamma=2, var=30)

In [ ]:
gen_ldr_budget = dc_opf_ldr_budget(demand,var_30,2)
evaluate_scenarios(gen_ldr_budget,scen_30)

Set parameter Threads to value 1
obj = 7112.5
Status = optimal
dispatch = [150. 100.  50.] [0.         0.16666667 0.83333333]


# Evaluate LDR dispatch (Gamma=1, var=10)

In [ ]:
gen_ldr_budget = dc_opf_ldr_budget(demand,var_10,1)
evaluate_scenarios(gen_ldr_budget,scen_10)

# Evaluate LDR dispatch (Gamma=1, var=20)

In [ ]:
gen_ldr_budget = dc_opf_ldr_budget(demand,var_20,1)
evaluate_scenarios(gen_ldr_budget,scen_20)

# Evaluate LDR dispatch (Gamma=1, var=30)

In [ ]:
gen_ldr_budget = dc_opf_ldr_budget(demand,var_30,1)
evaluate_scenarios(gen_ldr_budget,scen_30)

# Stochastic programming

We use the index $w$ for scenarios and $\pi_w$ for scenario probability. First stage decision are $g_n$ while second stage decisions are $r_{nw}$. The two-stage optimization problem is formulated as follows:

$$
\begin{align}
\underset{g_n,r^u_{nw}, r^d_{nw}}{\min} \quad &  \sum_w \pi_w\left( \sum_n c_n g_n + c^u_n r^u_{nw} - c^d_n r^d_{nw}\right)  \\
\text{s.t.} \quad & \sum_n g_n+r^u_{nw}-r^d_{nw} = \sum_n \hat{d}_n - \overline{e}_n\eta_{nw}, \quad \forall w \\
& 0 \leq g_n + r^u_{nw} - r^d_{nw} \leq \overline{g}_n, \quad \forall n,w\\
& -\overline{f}_l \leq \sum_n b_{ln}(g_n +r^u_{nw} - r^d_{nw} -\hat{d}_n + \overline{e}_n \eta_{nw}) \leq \overline{f}_l, \quad \forall l,w \\
& r^u_{nw}, r^d_{nw}\geq 0
\end{align}
$$

In [ ]:
def sto(demand,var,nscen):
  # generate scenarios
  eta = np.zeros((nbus,nscen))
  for n in range(nbus):
    for s in range(nscen):
      eta[n,s] = var[n]*np.random.uniform(-1,1)
  # model
  m = gp.Model()  
  # variables
  gen = m.addMVar(nbus)
  res_up = m.addMVar((nbus,nscen))
  res_do = m.addMVar((nbus,nscen))
  # objective function
  m.setObjective(cost_bus @ gen + sum(cost_up_bus @ res_up[:,s] for s in range(nscen))/nscen - sum(cost_do_bus @ res_do[:,s] for s in range(nscen))/nscen, GRB.MINIMIZE)
  # constraints
  m.addConstrs(gen.sum() + res_up[:,s].sum() - res_do[:,s].sum() == demand.sum() - eta[:,s].sum() for s in range(nscen))
  m.addConstrs(gen + res_up[:,s] - res_do[:,s] >= 0 for s in range(nscen))
  m.addConstrs(gen + res_up[:,s] - res_do[:,s] <= pmax_bus for s in range(nscen))
  m.addConstrs(ptdf @ gen + ptdf @ res_up[:,s] - ptdf @ res_do[:,s] >= -cap_line + ptdf @ demand - ptdf @ eta[:,s]  for s in range(nscen))
  m.addConstrs(ptdf @ gen + ptdf @ res_up[:,s] - ptdf @ res_do[:,s] <= cap_line + ptdf @ demand - ptdf @ eta[:,s] for s in range(nscen))
  # solve
  m.setParam('OutputFlag',0)  
  m.optimize()
  status(m)     
  # results
  print('dispatch =',gen.X)  
  return gen.X

# Evaluate stochastic dispatch with var=10

In [ ]:
gen_sto = sto(demand,var_10,100)
evaluate_scenarios(gen_sto,scen_10)

Status = optimal
dispatch = [150.         145.80915817   3.94554097]
minimum cost = 6344.741221540553
average cost = 6818.021480238825
maximum cost = 7575.370556356244


# Evaluate stochastic dispatch with var=20

In [ ]:
gen_sto = sto(demand,var_20,100)
evaluate_scenarios(gen_sto,scen_20)

Status = optimal
dispatch = [150.         143.87648489   5.52503214]
minimum cost = 5865.892123310881
average cost = 6864.474441251858
maximum cost = 8426.23269217924


# Evaluate stochastic dispatch with var=30

In [ ]:
gen_sto = sto(demand,var_30,100)
evaluate_scenarios(gen_sto,scen_30)

Status = optimal
dispatch = [150.         144.11270089   9.68333628]
minimum cost = 5470.323846457206
average cost = 6920.954638364649
maximum cost = 9269.528581572671
